W ramach budowy systemu śledzenia bil wykorzystano nagranie pochodzące z serwisu YouTube, dostępne pod [linkiem](https://www.youtube.com/watch?v=9awGUjKVewk). Film prezentuje fragment rozgrywki z kamery umieszczonej nad stołem bilardowym. Przed przystąpieniem do prac nagranie spowolniono do 20% oryginalnej prędkości oraz wycięto ścieżkę dźwiękową. Na początku zaimportowano niezbędne paczki oraz zdefiniowano sposob wczytywania i wyśwetlania filmu. Przetwarzanie odbywa się w pętli, która w ramach danej iteracji rozpatruje jedną klatke nagrania.

In [61]:
import numpy as np
import cv2

Klasa reprezentujaca bilę. Atrybuty x oraz y oznaczają współrzędne bili.

In [62]:
class Ball:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.color = None
    
    
    def dist(self, x, y):
        return np.sqrt((self.x - x) ** 2 + (self.y - y) ** 2)
    
    def set_position(self, x, y):
        self.x = x
        self.y = y
        
    def delete(self):
        self.set_position(None, None)
    
    def same_position(self, x, y):
        if self.x is None and self.y is None:
            return False
        return self.dist(x, y) < 1

Śledzenie rozpoczęto od ekstrkacji bil. Aby zredukować szum posłużono się filtrem Gaussa wyposażonym w jadro o rozmiarze 7x7. Kolejnym krokiem była konwersja klatki z oryginalnej przestrzeni barw na HSV. Następnie na podstawie wartości pikseli odpowiadajacych zielonej powierzchni stołu bilardowego utworzono maskę. Za jej pomocą wyodrębniono zioloną powierzchnię, a następnie dokonano transformacji obrazu do skali szarości, otrzymując jasny wykrywany obiekt na ciemnym tle. Rezultaty obu operacji zobrazowano poniżej:

![Ekstrakcja stołu](img/ekstrakcja_stołu.png)

![Ekstrakcja stołu w sklai szarości](img/stół_w_skali_szarości.png)

Kolejno zastosowano operację progowania z algorytmem OTSU, który adaptacyjnie dobiera wartość progu, minimalizując wariancję intensywności nowo powstałych klas. Zbinaryzowany obraz poddano dylatacji, aby poprawić jakość detekcji, zwiększajac rozmiar wykrytego obiektu. Efekt zaprezentowano poniżej:

![Dylatacja stołu](img/stół_po_dylatacji.png)

In [63]:
def mask_frame(frame, mask):
    new_frame = np.zeros_like(frame, np.uint8)
    mask = mask > 0
    new_frame[mask] = frame[mask]
    return new_frame

In [64]:
def get_billiard_balls(frame):
    gauss_blur_frame = cv2.GaussianBlur(frame, ksize=(7, 7), sigmaX=0) # 1.41
    hsv_frame = cv2.cvtColor(gauss_blur_frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_frame, (36, 100, 100), (70, 255, 255))
    masked_frame = mask_frame(gauss_blur_frame, mask)
    grey_frame = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
    _, binary_frame = cv2.threshold(grey_frame, 128, 255, cv2.THRESH_BINARY|cv2.THRESH_OTSU)
    return cv2.dilate(binary_frame, np.ones((1, 1), np.uint8))

In [65]:
def mark_balls(frame, billiard_balls):
    simple_blob_detector = cv2.SimpleBlobDetector_create()
    keypoints = simple_blob_detector.detect(billiard_balls)
    marker_color = (255, 0, 0)
    keypoints_frame = cv2.drawKeypoints(image=frame, keypoints=keypoints, outImage=np.array([]), color=marker_color, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return keypoints_frame, keypoints

Wykrywanie koloru bili rozpoczęto od zebrania wszystkich jej pikseli. Wykorzystano algorytm rozrostu zaprezentowany na labratoriach. Następnie obliczany jest średni kolor z wyodrębnionego fragmentu, który służy do zidentyfikowania koloru bili. Te same funkcje są również używane w dalszym przetwarzaniu kolejnych klatek, gdzie na podstawie punktów kluczowych wyznaczamy nowo rozpoznane bile, poruszone bile oraz rozpoznajemy ich kolor. 

![Ekstrakcja stołu](img/czerwona_bila.png)

In [66]:
def find_neighbours(img, regions, y, x):
    neighbours = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    c_neighbours = []
    for dy, dx in neighbours:
        ny, nx = y + dy, x + dx

        if ny < 0 or ny >= img.shape[0] or nx < 0 or nx >= img.shape[1]:
            continue

        if regions[ny, nx] > 0:
            continue

        if img[ny, nx] > 0:
            continue

        c_neighbours.append((ny, nx))

    return c_neighbours

def get_ball_mask(img, x, y):
    mask = np.zeros(img.shape)
    mask[y, x] = 1

    c_neighbours = find_neighbours(img, mask, y, x)
    for ny, nx in c_neighbours:
        mask[ny, nx] = 1

    while len(c_neighbours) > 0:
        new_neighbours = []
        for ny, nx in c_neighbours:
            i_new_neighbours = find_neighbours(img, mask, ny, nx)
            for _ny, _nz in i_new_neighbours:
                mask[_ny, _nz] = 1

            new_neighbours.extend(i_new_neighbours)

        c_neighbours = new_neighbours

    return mask


def hsv_2_color_name(h, s, v):
    assignment = [
        (v < 0.3, 'black'),
        (s < 0.3 and v > 0.65, 'white'),
        ((h < 20 or h > 351) and s > 0.7 and v > 0.3, 'red'),
        (29 < h < 64 and s > 0.15 and v > 0.3, 'yellow'),
        (64 < h < 80 and s > 0.15 and v > 0.3, 'green'),
        (80 < h < 255 and s > 0.15 and v > 0.3, 'blue'),
        (20 < h < 29 and s > 0.15 and 0.3 < v < 0.75, 'brown')
    ]

    for a, color in assignment:
        if a:
            return color
    return 'pink'

In [67]:
def count_hsv_stats(hsv):
    h, s, v = cv2.split(hsv)
    new_h = np.mean(h[h > 0])
    new_s = np.mean(s[s > 0]) / 255
    new_v = np.mean(v[v > 0]) / 255
    return new_h, new_s, new_v

def colors_of_keypoints(keypoints, frame, billiard_balls):
    keypoints_to_colors = []
    for keypoint in keypoints:
        x, y = keypoint.pt[0], keypoint.pt[1]
        mask = get_ball_mask(billiard_balls, int(x), int(y))
        masked = mask_frame(frame, mask)
        hsv = cv2.cvtColor(masked, cv2.COLOR_BGR2HSV)
        h, s, v = count_hsv_stats(hsv)
        color = hsv_2_color_name(h, s, v)
        keypoints_to_colors.append(((x, y), color))
    return keypoints_to_colors

Kolejne funkcje służą do śledzenia bil. Jeśli w aktualnej klatce istnieje punkt kluczowy, którego odległość jest mniejsza od 1 to uznajemy go za tą bilę i przypisujemy jej tą pozycję. W drugim przypadku kiedy nie znajdziemy takiego punktu kluczowego w tym samym miejscu to szukamy punktu kluczowego w tym samym kolorze. Na jego podstawie wyznaczamy nową pozycję dla bili, która się poruszyła. W ostatecznym przypadku jeśli zostaje jedna bila niedopasowana i jeden nieprzydzielony punkt kluczowy to pozycja bili zotaje wyznaczona na podstawie tego punktu kluczowego. 

In [68]:
def find_keypoint_with_same_position(ball, keypoints):
    for keypoint in keypoints:
        if ball.same_position(*keypoint.pt):
            ball.set_position(*keypoint.pt)    
            return keypoint

def get_moved_balls_keypoints(balls, keypoints):
    moved_balls = []
    moved_keypoints = list(keypoints[:])
    for ball in balls:
        found_keypoint = find_keypoint_with_same_position(ball, keypoints)
        if found_keypoint is not None:
            moved_keypoints.remove(found_keypoint)
        else:
            moved_balls.append(ball)
    return moved_balls, moved_keypoints

def find_keypoint_with_same_color(ball, keypoints):
    for keypoint in keypoints:
        if ball.color == keypoint[1]:
            ball.set_position(*keypoint[0])    
            return keypoint
        
def get_mismatched_balls_keypoints(moved_balls, keypoints):
    mismatched_balls = []
    for ball in moved_balls:
        found_keypoint = find_keypoint_with_same_color(ball, keypoints)
        if found_keypoint is not None:
            keypoints.remove(found_keypoint)
        else:
            mismatched_balls.append(ball)
            
    return mismatched_balls, keypoints

def process_mismatched_balls_keypoints(mismatched_balls, keypoints):
    if len(mismatched_balls) == 1 and len(keypoints) == 1:
        ball = mismatched_balls[0]
        keypoint = keypoints[0]
        x, y = keypoint[0]
        ball.x, ball.y = x, y
        
    for ball in mismatched_balls:
        ball.delete()        
        
def update_balls(balls, keypoints, frame, processed_frame):
    moved_balls, moved_keypoints = get_moved_balls_keypoints(balls, keypoints)
    keypoints_to_colors = colors_of_keypoints(moved_keypoints, frame, processed_frame)
    mismatched_balls, mismatched_keypoints = get_mismatched_balls_keypoints(moved_balls, keypoints_to_colors)
    process_mismatched_balls_keypoints(mismatched_balls, mismatched_keypoints)

In [69]:
def assign_colors(balls, billiard_balls, frame):
    for ball in balls:
        mask = get_ball_mask(billiard_balls, int(ball.x), int(ball.y))
        masked_frame = mask_frame(frame, mask)
        hsv = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2HSV)
        h, s, v = count_hsv_stats(hsv)
        color = hsv_2_color_name(h, s, v)
        ball.color = color
    return balls

def show_description(frame, balls):
    for ball in balls:
        if ball.x is not None and ball.y is not None:
            x = int(ball.x)
            y = int(ball.y)
            content = ball.color+' '+str(x)+' '+str(y)
            cv2.putText(frame, content, (x, y), cv2.FONT_HERSHEY_TRIPLEX, 0.4, (253, 90, 159), 1, cv2.LINE_AA)
    return frame

Przetworzenie nagrania w każdej klatce rozpoczyna się od  przekształceń morfologicznych i uzyskania punktów kluczowych. W przypadku pierwszej klatki, następuje utworzenie bil wraz z przypisaniem im kolorów, na podstawie wyodrębnionych regionów. Jeśli jest to kolejna klatka, to zostaje uaktualniona pozycja zaincjowanych na początku bil, na podstawie punktów kluczowych z aktualnej klatki zgodnie z procedurą śledzenia bil. 

In [70]:
cap = cv2.VideoCapture("./bilard.mp4")
cv2.namedWindow('Balls tracking', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Balls tracking', 1400, 800)
first = True

while cap.isOpened():
    _, frame = cap.read()
    billiard_balls = get_billiard_balls(frame)
    keypoints_frame, keypoints = mark_balls(frame, billiard_balls)

    if first:
        first = False
        balls = [Ball(k.pt[0], k.pt[1]) for k in keypoints]
        balls = assign_colors(balls, billiard_balls, frame)

    update_balls(balls, keypoints, frame, billiard_balls)
    keypoints_frame = show_description(keypoints_frame, balls)
    cv2.imshow('Balls tracking', keypoints_frame)
    if cv2.waitKey(1) == ord('q'):
         cap.release()
         break
cv2.destroyAllWindows()